In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!pip install transformers gradio gtts torch

In [ ]:
!pip install gtts

In [ ]:
!pip install torch torchvision transformers datasets gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install pytesseract

In [ ]:
!pip install torch pillow transformers pytesseract gradio gTTS ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers pillow pytesseract gradio gtts
!apt-get install tesseract-ocr -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (15.6 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
#final with feedback

import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import pytesseract
from gtts import gTTS
import gradio as gr
import hashlib
import json
import os

# Load the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Check if GPU is available and use it
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# File to store user feedback (image hash to caption mapping)
FEEDBACK_FILE = "feedback_data.json"

# Safely load feedback data if the file exists
try:
    if os.path.exists(FEEDBACK_FILE):
        with open(FEEDBACK_FILE, "r") as f:
            feedback_data = json.load(f)
    else:
        feedback_data = {}
except json.JSONDecodeError:
    feedback_data = {}
    print("Warning: Could not decode feedback_data.json. Starting fresh.")

def hash_image(img):
    """Generate a hash for the image to use as a unique identifier."""
    img_bytes = img.tobytes()
    return hashlib.sha256(img_bytes).hexdigest()

def generate_caption_with_feedback(img):
    """
    Generate a caption for the given image, allowing feedback for correction.

    Args:
        img (numpy.ndarray): The input image array.

    Returns:
        tuple: The generated or retrieved caption, audio for playback, and image hash.
    """
    try:
        # Convert the image to a PIL Image
        img_input = Image.fromarray(img)

        # Hash the image to check for existing feedback
        img_hash = hash_image(img_input)

        # If a corrected caption exists, use it
        if img_hash in feedback_data:
            caption = feedback_data[img_hash]
        else:
            # Generate a descriptive caption
            inputs = processor(images=img_input, return_tensors="pt").to(device)
            outputs = model.generate(**inputs)
            caption = processor.decode(outputs[0], skip_special_tokens=True)

            # Extract text from the image using pytesseract
            detected_text = pytesseract.image_to_string(img_input).strip()
            if detected_text:
                caption += f" Detected text in the image: {detected_text}"

        # Generate speech from the caption using gTTS
        tts = gTTS(text=caption, lang="en")
        audio_file = "caption_feedback.mp3"
        tts.save(audio_file)

        return caption, audio_file, img_hash
    except Exception as e:
        return f"Error generating caption: {str(e)}", None, None

def save_feedback(img_hash, corrected_caption):
    """
    Save corrected captions provided by the user.

    Args:
        img_hash (str): The unique hash for the image.
        corrected_caption (str): The user-provided corrected caption.
    """
    feedback_data[img_hash] = corrected_caption
    with open(FEEDBACK_FILE, "w") as f:
        json.dump(feedback_data, f)

# Create a Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# SceneScribe: AI Powered Image Captioning")

    with gr.Row():
        img_input = gr.Image(label="Upload an Image", type="numpy")
        caption_output = gr.Textbox(label="Generated Caption")
        audio_output = gr.Audio(label="Audio Caption")

    corrected_caption_input = gr.Textbox(label="Provide Corrected Caption (if incorrect)")
    feedback_button = gr.Button("Submit Corrected Caption")

    feedback_status = gr.Textbox(label="Feedback Status", interactive=False)

    generate_button = gr.Button("Generate Caption")

    def generate_and_display(img):
        caption, audio, img_hash = generate_caption_with_feedback(img)
        return caption, audio, img_hash

    def handle_feedback(img, corrected_caption):
        if img is None:
            return "Error: No image provided."
        if not corrected_caption.strip():
            return "Error: Corrected caption cannot be empty."

        try:
            img_hash = hash_image(Image.fromarray(img))
            save_feedback(img_hash, corrected_caption)
            return f"Corrected caption saved: {corrected_caption}"
        except Exception as e:
            return f"Error saving feedback: {str(e)}"

    generate_button.click(
        generate_and_display,
        inputs=[img_input],
        outputs=[caption_output, audio_output, feedback_status],
    )

    feedback_button.click(
        handle_feedback,
        inputs=[img_input, corrected_caption_input],
        outputs=feedback_status,
    )

# Add a friendly warning if GPU is not available
if device == "cpu":
    print("Warning: GPU not detected. Running on CPU may be slower.")

# Launch the app
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b56be53524bc676def.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
